In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [6]:
import pandas as pd

COMPANY = 'NVDA'
# read the csv
data = pd.read_csv("/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/data_NVDA.csv")
data.head()
# drop nulls
data.isnull().sum()
# select only required columns
UNWANTED_COLS = ["volume"]

# filter columns
df = data[[col for col in list(data.columns) if col not in UNWANTED_COLS]]
df.head()
# conver the data to pandas date time
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", drop = True, inplace = True)

<ipython-input-6-7f88940a0d21>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = pd.to_datetime(df["date"])


In [7]:
from sklearn.preprocessing import MinMaxScaler
import pickle
import os

MODEL_SAVE_PATH = "/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/"

COLUMNS = list(df.columns)
print("Columns", COLUMNS)

# fit the model
scaler = MinMaxScaler().fit(df[COLUMNS])
# save the model
with open(os.path.join(MODEL_SAVE_PATH, f"{COMPANY}scalermodel_GRU"), "wb") as tesla_model:
    pickle.dump(scaler, tesla_model)
    print("Model Saved")

# fot on data
df[COLUMNS] = scaler.transform(df[COLUMNS])

Columns ['open', 'high', 'low', 'close']
Model Saved


<ipython-input-7-16689d992bb8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[COLUMNS] = scaler.transform(df[COLUMNS])


In [14]:
# define the plsit ratio
SPLIT_RATIO = 0.2

# train length
train_length = int(len(df) * (1 - SPLIT_RATIO))
print("train length", train_length)

train_data = df[:train_length]
test_data = df[train_length:]
print("train shape", train_data.shape)
print("test shape", test_data.shape)

train length 1007
train shape (1007, 4)
test shape (252, 4)


In [16]:
train_data.to_csv("/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/train_NVDA_GRU.csv",index="TRUE")
test_data.to_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/test_NVDA_GRU.csv',index="TRUE")

In [3]:
COMPANY = "NVDA"
train_data= pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/train_NVDA.csv')
test_data=pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/test_NVDA.csv')

In [19]:
train_data= pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/train_NVDA_GRU.csv')
test_data=pd.read_csv('/content/drive/Shareddrives/1:1_Krishna_Vijay/Dataset/test_NVDA_GRU.csv')

In [4]:
import numpy as np

def prepare_seq(data,seq_len):    #eg seq=len = 50
    sequences,labels = [],[]
    #len of data is 1353 then 1353 - 50 is the range
    for i in range(len(data) - seq_len):
        #eg 0 to 50
        x_temp = data.iloc[i:i+seq_len].values
        #stores 50th value
        y_temp = data.iloc[i+seq_len].values

        sequences.append(x_temp)
        labels.append(y_temp)

    return np.array(sequences), np.array(labels)

Training of model

In [5]:
# define the sequence length
SEQ_LENGTH = 50

# process the dataset
train_sequences,train_labels = prepare_seq(train_data, SEQ_LENGTH)
test_sequences,test_labels = prepare_seq(test_data, SEQ_LENGTH)
print(train_sequences.shape, test_sequences.shape)

(957, 50, 4) (202, 50, 4)


creating model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Dropout
from tensorflow.keras.optimizers import Adam

# define the learning rate and epochs
LR = 0.001
EPOCHS = 50

# build the model
model = Sequential()
model.add(GRU(units=50,return_sequences = True, input_shape=(train_sequences.shape[1],train_sequences.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units = 50))
model.add(Dense(4))

# compile the model
model.compile(optimizer=Adam(learning_rate = LR), loss='mean_squared_error', metrics=['mean_squared_error'])

training model

In [7]:
# train the model
model.fit(train_sequences,train_labels, epochs=EPOCHS, validation_data = (test_sequences,test_labels),verbose = 'auto')

Epoch 1/50
30/30 [==============================] - 8s 95ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0062 - val_mean_squared_error: 0.0062
Epoch 2/50
30/30 [==============================] - 3s 85ms/step - loss: 1.9134e-04 - mean_squared_error: 1.9134e-04 - val_loss: 0.0019 - val_mean_squared_error: 0.0019
Epoch 3/50
30/30 [==============================] - 2s 65ms/step - loss: 1.0348e-04 - mean_squared_error: 1.0348e-04 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 4/50
30/30 [==============================] - 1s 49ms/step - loss: 9.0149e-05 - mean_squared_error: 9.0149e-05 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 5/50
30/30 [==============================] - 1s 49ms/step - loss: 8.8916e-05 - mean_squared_error: 8.8916e-05 - val_loss: 0.0020 - val_mean_squared_error: 0.0020
Epoch 6/50
30/30 [==============================] - 1s 48ms/step - loss: 8.5762e-05 - mean_squared_error: 8.5762e-05 - val_loss: 0.0014 - val_mean_squared_error: 0

Save best model

In [27]:
import os
import pickle

COMPANY = 'NVDA'
SAVE = True
GRU_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/"

# save the model if needed
#if SAVE:
#    model.save("/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/best_model_NVDA_GRU.h5")
#    print("Model is saved")

with open("/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/best_model_NVDA_GRU.h5", 'wb') as file:
  pickle.dump(model, file)

In [9]:
SAVE = True
#LSTM_MODEL_PATH = "/content/drive/Shareddrives/ResearchPrograms/TimeSeriesProject/MultiCompanyStocks/Models"

# save the model if needed
if SAVE:
    model.save("/content/drive/Shareddrives/1:1_Krishna_Vijay/Results/Models/best_model_NVDA_GRU.h5")
    print("Model is saved")

Model is saved


Making predictions

In [ ]:
# make predictions
predictions = model.predict(test_sequences)
# inverse tranform the predictions
inverse_predictions = scaler.inverse_transform(predictions)

7/7 [==============================] - 1s 13ms/step


NameError: name 'scaler' is not defined

making eval function

In [ ]:
# create a dataframe frame using model predictions
PRED_COLUMNS = ['open_predicted','high_predicted','low_predicted','close_predicted']
# create the dataframe
pred_df = pd.concat([df.iloc[-predictions.shape[0]:], pd.DataFrame(inverse_predictions, columns = PRED_COLUMNS, index = df.iloc[-predictions.shape[0]:].index)], axis = 1)
pred_df[COLUMNS] = scaler.inverse_transform(pred_df[COLUMNS])
pred_df.head()

In [ ]:
from sklearn.metrics import mean_squared_error

def evaluations(cols, dataframe = pred_df):
    for column in cols:
        print("{}".format(column))
        pred_col = column.lower() + "_predicted"
        mse = mean_squared_error(dataframe[column].values, dataframe[pred_col].values)
        print("Max and Min: {} and {}".format(dataframe[column].max(), dataframe[column].min()))
        print("MSE:", mse)
        print("RMSE:{}\n".format(np.sqrt(mse)))

        plt.figure(figsize = (10,6))
        plt.plot(dataframe[column], label = "actual")
        plt.plot(dataframe[pred_col], label = "predicted")
        plt.xlabel("Time")
        plt.ylabel(f"Actual and Predicted:{column}")
        plt.title(f"Actual and Predictions Comparison: {column}")
        plt.legend(loc="best")
        plt.grid(True)
        plt.show()
        print("\n\n")

evaluating

In [ ]:
# make evaluations
evaluations(COLUMNS)